In [38]:
import cv2
import numpy as np

### 打开关闭窗口

In [1]:
#创建窗口
cv2.namedWindow('windows',cv2.WINDOW_NORMAL)
#修改窗口大小
cv2.resizeWindow('windows',800,600)
#展示名字为window的窗口
cv2.imshow('windows',0)
#等待按键输入，最多delay 5000ms
key = cv2.waitKey(0)
#key 是int型，最少是16位，但是ascii码是8位
if key & 0xFF == ord('q'):
    print("exit")
    cv2.destroyAllWindows()

exit


### 读取图片并打开

In [2]:
def cv_show(name,img):
    cv2.imshow(name,img)
    key = cv2.waitKey(0)
    if key == ord('q'):
        cv2.destroyAllWindows()

#读相对目录下的某个照片
cat = cv2.imread('./picture/cat.jpeg')

#窗口打开图片
cv_show('cat',cat)

### 保存图片

In [5]:
def cv_catch(num,name):
    str_num = str(num)
    catalog = f"""./picture/{str_num}.png"""
    print("save in"+catalog)
    cv2.imwrite(catalog,name)

In [ ]:
cv2.imshow('cat',cat)
num = 0
while True:
    key = cv2.waitKey(0)
    #每次按下's'键后，保存图片并num+1
    if key == ord('s'):       
        cv_catch(num,cat)
        num += 1 
    #每次按下'q'键后，退出所有窗口
    elif key == ord('q'):
        num = 0
        cv2.destroyAllWindows()
        break

### 视频采集

In [4]:
#打开目录的指定视频

cv2.namedWindow('video',cv2.WINDOW_NORMAL)


#打开摄像头
#cap = cv2.VideoCapture(0)
#cv2.resizeWindow('video',640,480)

#打开视频
cap = cv2.VideoCapture('./video/koen.mp4')
cv2.resizeWindow('video',720,1280)

#循环读每一帧
while True:
    #读一帧，返回标记
    ret, frame = cap.read()
    #根据ret判断
    if not ret:
        break
    #显示数据
    cv2.imshow('video',frame)
    
    #隔100ms检测一下
    key = cv2.waitKey(16)
    if key == ord('q'):
        cv2.destroyAllWindows()
        break
    cap.release
    


### 识别视频中的人脸

In [35]:
def detector_class(frame,Classifier):
    #得到灰阶图
    gray = cv2.cvtColor(frame, code=cv2.COLOR_BGR2GRAY)
    
    #检测特征物
    detector_frame = Classifier.detectMultiScale(gray,
                                                 scaleFactor = 1.1,
                                                 minNeighbors = 10)
    #在原始画面红框圈出特征物
    for x,y,h,w in detector_frame:
        cv2.rectangle(frame,
                    pt1 = (x,y),
                    pt2=(x+w,y+h),
                    color=[0,0,255],thickness=2
                    ) 

In [37]:
cap = cv2.VideoCapture(0)
cv2.namedWindow('video',cv2.WINDOW_NORMAL)
cv2.resizeWindow('video',640,480)
#导入特征文件
face_detector = cv2.CascadeClassifier('./classfile/haarcascade_frontalface_alt.xml')
num = 0
while True:
    #ret为是否读取标志位
    ret,frame = cap.read() 
    #r如果读不到，则退出和释放
    if not ret:
        break
    #圈出识别物
    detector_class(frame,face_detector)
    cv2.imshow('video',frame)
    #根据帧数取整设置延时
    key = cv2.waitKey(1000//60)
    #键入S抓取视频图片
    if key == ord('s'):       
        cv_catch(num,frame)
        num += 1 
    #键入Q则关闭窗口和释放
    elif key == ord('q'): 
        num = 0
        print("Exit now")
        break   
cv2.destroyAllWindows()
cap.release()
    


Exit now


### 加入ssd模型做人脸识别

In [86]:
cap = cv2.VideoCapture(0)
w = 640
h = 480
cv2.namedWindow('video',cv2.WINDOW_NORMAL)
cv2.resizeWindow('video',w,h)
#导入特征文件
face_detector = cv2.CascadeClassifier('./classfile/haarcascade_frontalface_alt.xml')

model_file = r"./classfile/ssd/deploy.prototxt.txt"
pretrained_weights = r"./classfile/ssd/res10_300x300_ssd_iter_140000_fp16.caffemodel"
# 加载预训练的SSD模型和权重
net = cv2.dnn.readNetFromCaffe(model_file, pretrained_weights)


num = 0
while True:
    #ret为是否读取标志位
    ret,frame = cap.read() 
    #r如果读不到，则退出和释放
    if not ret:
        break
    #将图像的大小调整为300x300,并创建blob
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (512, 512)), 1.0,
                            (512, 512), (104.0, 177.0, 123.0))       
    # 输入blob到网络中
    net.setInput(blob)
    #np.squeeze减少大小为1的维度
    detections = np.squeeze(net.forward())
    # 遍历检测结果
    for i in range(detections.shape[0]):
        confidence = detections[i, 2]  # 获取置信度
        
        if confidence > 0.3:
            # 通过检测结果计算人脸的边界框坐标,调整3:的值来调整识别精度，原始值为7
            box = detections[i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            # 构建标签字符串
            label = f"Face: {confidence * 100:.2f}%"
            # 绘制边界框和标签
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
            y = startY - 15 if startY - 15 > 15 else startY + 15
            cv2.putText(frame, label, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    
    cv2.imshow('video',frame)
    #根据帧数取整设置延时
    key = cv2.waitKey(1000//90)
    #键入S抓取视频图片
    if key == ord('s'):       
        cv_catch(num,frame)
        num += 1 
    #键入Q则关闭窗口和释放
    elif key == ord('q') : 
        num = 0
        print("Exit now")
        break   
cv2.destroyAllWindows()
cap.release()
    


Exit now
